In [1]:
PROJECT_DIR=`git rev-parse --show-toplevel`
cd $PROJECT_DIR/kmer-mlst

# Get MLST profiles

In [2]:
wget https://pubmlst.org/data/alleles/senterica/aroC.tfa

--2020-01-18 15:57:29--  https://pubmlst.org/data/alleles/senterica/aroC.tfa
Resolving pubmlst.org (pubmlst.org)... 129.67.24.31
Connecting to pubmlst.org (pubmlst.org)|129.67.24.31|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 511143 (499K)
Saving to: ‘aroC.tfa’

aroC.tfa            100%[===================>] 499.16K   856KB/s    in 0.6s    

2020-01-18 15:57:30 (856 KB/s) - ‘aroC.tfa’ saved [511143/511143]



In [7]:
du -sh aroC.tfa

500K	aroC.tfa


Pick first 10 alleles from this file.

In [12]:
head -n 100 aroC.tfa > aroC-10.fasta

Align everything with clustalomega.

In [58]:
conda run --name clustalo clustalo -i aroC-10.fasta -o aroC-10.aligned.fasta --force
conda run --name clustalo clustalo -i aroC-10.fasta --outfmt phylip -o aroC-10.aligned.phy --force

Also build phylogenetic tree.

In [61]:
conda run --name phyml phyml -i aroC-10.aligned.phy | head -n 3



. Command line: phyml -i aroC-10.aligned.phy 


In [63]:
cat aroC-10.aligned.phy_phyml_tree.txt

(aroC_9:0.00635288,aroC_3:0.00206180,(aroC_6:0.00208228,(aroC_2:0.00000001,(aroC_1:0.00866459,(aroC_10:0.00000001,(aroC_4:0.00000001,(aroC_5:0.00423962,(aroC_8:0.00417796,aroC_7:0.00000001)0.000000:0.00000001)0.962873:0.00207799)0.977224:0.00207798)0.791440:0.00209504)0.791428:0.00209499)0.879313:0.00209523)0.991691:0.00428820);


In [95]:
python -c '
from Bio import AlignIO
align = AlignIO.read("aroC-10.aligned.fasta", "fasta")
print(align)
'

SingleLetterAlphabet() alignment with 10 rows and 501 columns
GTTTTTCGCCCGGGACACGCGGATTACACCTATGAGCAGAAATA...GGT aroC_1
GTTTTTCGTCCGGGACACGCGGATTACACCTATGAGCAGAAATA...GGT aroC_2
GTTTTTCGTCCGGGACACGCGGATTACACCTATGAGCAGAAATA...GGT aroC_3
GTTTTTCGTCCGGGACACGCGGATTACACCTATGAGCAGAAATA...GGT aroC_4
GTTTTTCGTCCGGGACACGCGGATTACACCTATGAGCAGAAATA...GGT aroC_5
GTTTTTCGTCCGGGACACGCGGATTACACCTATGAGCAGAAATA...GGT aroC_6
GTTTTTCGTCCGGGACACGCGGATTACACCTATGAGCAGAAATA...GGT aroC_7
GTTTTTCGTCCGGGACACGCGGATTACACCTATGAGCAGAAATA...GGT aroC_8
GTTTTTCGTCCGGGACACGCGGATTACACCTATGAGCAGAAATA...GGT aroC_9
GTTTTTCGTCCGGGACACGCGGATTACACCTATGAGCAGAAATA...GGT aroC_10


In [97]:
python -c '
from Bio import AlignIO
align = AlignIO.read("aroC-10.aligned.fasta", "fasta")

def check_freq(str):
    freq = {}
    min=1000
    min_char = None
    for c in set(str):
       freq[c] = str.count(c)
       if freq[c] < min:
           min = freq[c]
           min_char = c
    return (freq, min_char)

snp_col_row = {}
for i in range(align.get_alignment_length()):
    col = align[:, i]
    (cf, min_char) = check_freq(col)
    
    if len(cf) > 1:
        left_edge = max(0, i - 15)
        right_edge = min(align.get_alignment_length(), left_edge + 31)
        #left_edge = max(0, i - 10)
        #right_edge = min(align.get_alignment_length(), left_edge + 21)
        kmer_slice = align[:, left_edge:right_edge]
        
        kmer_table = {}
        for j in kmer_slice:
            kmer_string = str(j.seq)
            if not kmer_string in kmer_table:
                kmer_table[kmer_string] = [j.id]
            else:
                kmer_table[kmer_string].append(j.id)
        
        print(f"Position {i}")
        for kmer in kmer_table:
            #kmer_with_snp = kmer[:i] + "[" + kmer[i] + "]" + kmer[i+1:]
            #print(f"\t{kmer_with_snp}")
            print(f"\t{kmer}")
            for id in kmer_table[kmer]:
                print(f"\t\t{id}")    
'

Position 8
	GTTTTTCGCCCGGGACACGCGGATTACACCT
		aroC_1
	GTTTTTCGTCCGGGACACGCGGATTACACCT
		aroC_2
		aroC_3
		aroC_4
		aroC_5
		aroC_6
		aroC_7
		aroC_8
		aroC_9
		aroC_10
Position 53
	GAAATACGGCCTGCGCGATTACCGCGGCGGT
		aroC_1
	GAAATACGGCCTGCGCGATTACCGTGGCGGT
		aroC_2
		aroC_4
		aroC_5
		aroC_6
		aroC_7
		aroC_8
		aroC_10
	GAAATACGGCCTGCGTGATTACCGTGGCGGT
		aroC_3
		aroC_9
Position 62
	CCTGCGCGATTACCGCGGCGGTGGACGTTCT
		aroC_1
	CCTGCGCGATTACCGTGGCGGTGGACGTTCT
		aroC_2
		aroC_4
		aroC_5
		aroC_6
		aroC_7
		aroC_8
		aroC_10
	CCTGCGTGATTACCGTGGCGGTGGACGTTCT
		aroC_3
		aroC_9
Position 119
	AGCGGCAGGGGCGATCGCCAAGAAATACTTG
		aroC_1
	AGCGGCAGGGGCGATCGCCAAGAAATACCTG
		aroC_2
		aroC_4
		aroC_5
		aroC_6
		aroC_7
		aroC_8
		aroC_10
	AGCGGCAGGGGCGATTGCCAAGAAATACCTG
		aroC_3
		aroC_9
Position 132
	ATCGCCAAGAAATACTTGGCGGAAAAGTTCG
		aroC_1
	ATCGCCAAGAAATACCTGGCGGAAAAGTTCG
		aroC_2
		aroC_4
		aroC_5
		aroC_6
		aroC_7
		aroC_8
		aroC_10
	ATTGCCAAGAAATACCTGGCGGAAAAGTTCG
		aroC_3
		aroC_9
Position 149
	GGCGGAAA

Awesome. I used these to try and build a biohansel scheme. Let's look at it now.

In [119]:
head mlst-biohansel-aroC10.fasta

>8-1
GTTTTTCGCCCGGGACACGCGGATTACACCT
>53-1
GAAATACGGCCTGCGCGATTACCGCGGCGGT
>53-2.1.1.1
CCTGCGTGATTACCGTGGCGGTGGACGTTCT
>62-1
CCTGCGCGATTACCGCGGCGGTGGACGTTCT
>62-2.1.1.1
CCTGCGTGATTACCGTGGCGGTGGACGTTCT


I've also created a metadata table mapping these SNP/SNV codes to MLST types (for aroC).

In [108]:
column -s$'\t' -t mlst-biohansel-metadata.tsv

subtype    mlst_aroC
1          1
2.1.1.1.1  3
2.1.1.1.2  9
2.1.1.2    6
2.1.2      2
2.2.1.1.1  7
2.2.1.1.2  8
2.2.1.1.3  5
2.2.1.2    4
2.2.2      10


I also have some fastq files we can use to test out.

In [121]:
du -sh test/fastq/*.fq

664M	test/fastq/aroC-11_GCF_000940975.1_ASM94097v1_genomic.fna.reads.fq
680M	test/fastq/aroC-5_GCF_000750435.1_ASM75043v1_genomic.fna.reads.fq
688M	test/fastq/aroC-8_GCF_000487135.1_ASM48713v1_genomic.fna.reads.fq
690M	test/fastq/aroC-9_GCF_000430975.2_ASM43097v2_genomic.fna.reads.fq


Let's try running biohansel on these files with the above scheme.

In [122]:
conda run --name biohansel /usr/bin/time -v hansel -s mlst-biohansel-aroC10.fasta test/fastq/*.fq -o biohansel-out \
    -M mlst-biohansel-metadata.tsv --low-cov-warning 15 --force

	Command being timed: "hansel -s mlst-biohansel-aroC10.fasta test/fastq/aroC-11_GCF_000940975.1_ASM94097v1_genomic.fna.reads.fq test/fastq/aroC-5_GCF_000750435.1_ASM75043v1_genomic.fna.reads.fq test/fastq/aroC-8_GCF_000487135.1_ASM48713v1_genomic.fna.reads.fq test/fastq/aroC-9_GCF_000430975.2_ASM43097v2_genomic.fna.reads.fq -o biohansel-out -M mlst-biohansel-metadata.tsv --low-cov-warning 15 --force"
	User time (seconds): 47.13
	System time (seconds): 4.73
	Percent of CPU this job got: 114%
	Elapsed (wall clock) time (h:mm:ss or m:ss): 0:45.27
	Average shared text size (kbytes): 0
	Average unshared data size (kbytes): 0
	Average stack size (kbytes): 0
	Average total size (kbytes): 0
	Maximum resident set size (kbytes): 65792
	Average resident set size (kbytes): 0
	Major (requiring I/O) page faults: 0
	Minor (reclaiming a frame) page faults: 27661
	Voluntary context switches: 128
	Involuntary context switches: 59814
	Swaps: 0
	File system inputs: 0
	File system outputs: 8
	Socket messag

Let's look at the results.

In [133]:
cat biohansel-out
echo
cut -f 1,4,7,16,17,19 biohansel-out | tee results-out.tsv | column -s$'\t' -t

sample	scheme	scheme_version	subtype	all_subtypes	kmers_matching_subtype	are_subtypes_consistent	inconsistent_subtypes	n_kmers_matching_all	n_kmers_matching_all_expected	n_kmers_matching_positive	n_kmers_matching_positive_expected	n_kmers_matching_subtype	n_kmers_matching_subtype_expected	file_path	avg_kmer_coverage	qc_status	qc_message	mlst_aroC
aroC-11_GCF00940975.1_ASM94097v1_genomic.fna.reads	mlst-biohansel-aroC10.fasta		2.2.1.1.3	2.2; 2.1.1.2; 2.2.1.1.3	2.2.1.1.3	False	['2.1.1.2', '2.2', '2.2.1.1.3']	3	3	3	3	1	2	['test/fastq/aroC-11_GCF_000940975.1_ASM94097v1_genomic.fna.reads.fq']	28.750	FAIL	"FAIL: 33.33% missing kmers; more than 5.00% missing kmers threshold. Okay coverage depth (38.0 >= 20.0 expected), but this may be the wrong serovar or species for scheme ""mlst-biohansel-aroC10.fasta"" | FAIL: Mixed subtypes found: ""2.1.1.2; 2.2; 2.2.1.1.3"". | FAIL: Unconfident Results Error 4: Subtype ""2.2.1.1.3"" was found, but kmers for nested hierarchical subtype(s) ""2.2.1.1; 2.2.1;

Looks like they failed because my scheme isn't complete. But the MLST results do match (other than type 11, which wasn't in my original scheme).

In [134]:
cat results-out.tsv

sample	subtype	are_subtypes_consistent	avg_kmer_coverage	qc_status	mlst_aroC
aroC-11_GCF00940975.1_ASM94097v1_genomic.fna.reads	2.2.1.1.3	False	28.750	FAIL	5
aroC-5_GCF00750435.1_ASM75043v1_genomic.fna.reads	2.2.1.1.3	True	38.667	FAIL	5
aroC-8_GCF00487135.1_ASM48713v1_genomic.fna.reads	2.2.1.1.2	True	38.600	FAIL	8
aroC-9_GCF00430975.2_ASM43097v2_genomic.fna.reads	2.1.1.1.2	True	28.000	FAIL	9


# Ignore biohansel and identify only lists of k-mers for each mlst allele

In [50]:
python -c '
from Bio import AlignIO
from Bio.Seq import Seq
align = AlignIO.read("aroC-10.aligned.fasta", "fasta")

def check_freq(str):
    freq = {}
    min=1000
    min_char = None
    for c in set(str):
       freq[c] = str.count(c)
       if freq[c] < min:
           min = freq[c]
           min_char = c
    return (freq, min_char)

table_of_kmers = {}
for i in range(align.get_alignment_length()):
    col = align[:, i]
    (cf, min_char) = check_freq(col)
    
    if len(cf) > 1:
        left_edge = max(0, i - 10)
        right_edge = min(align.get_alignment_length(), left_edge + 21)
        kmer_slice = align[:, left_edge:right_edge]
        
        for j in kmer_slice:
            kmer_string = str(j.seq)
            #kmer_seq_obj = Seq(kmer_string)
            #kmer_rc = str(kmer_seq_obj.reverse_complement())
            
            if not j.id in table_of_kmers:
                #table_of_kmers[j.id] = set([kmer_string, kmer_rc])
                table_of_kmers[j.id] = set([kmer_string])
            else:
                table_of_kmers[j.id].add(kmer_string)
                #table_of_kmers[j.id].add(kmer_rc)
                
for id in table_of_kmers:
    n = f"{id}.kmers"
    f = open(n, "w")
    print(f"Writing file {n}")
    for kmer in table_of_kmers[id]:
        f.write(f"{kmer}\n")
    f.close()
'

Writing file aroC_1.kmers
Writing file aroC_2.kmers
Writing file aroC_3.kmers
Writing file aroC_4.kmers
Writing file aroC_5.kmers
Writing file aroC_6.kmers
Writing file aroC_7.kmers
Writing file aroC_8.kmers
Writing file aroC_9.kmers
Writing file aroC_10.kmers


I've written my list of candidate kmers. Let's look at one file.

In [51]:
cat aroC_1.kmers
wc -l aroC_1.kmers

ACGGCCTGCGCGATTACCGCG
GCGAACCGGTATTTGACCGAC
TGATGAGCATCAATGCGGTGA
GTTTTTCGCCCGGGACACGCG
AGATGGGCGACATTCCGCTGG
AAAAGTTCGGCATCGAAATCC
GCCCCGATGCGGACAAACTTG
GCTGGACGAACTGATGCGCGC
GATTTAACGTGGTGGCGCTGC
TGGCGCTGCGCGGCAGCCAGA
GCGATTACCGCGGCGGTGGAC
GGACAAACTTGACGCGCTGGA
CAGGGGCGATCGCCAAGAAAT
CGTTCTTTTGCCCCGATGCGG
AAAAAGAGGGTGACTCCATCG
CAAGAAATACTTGGCGGAAAA
16 aroC_1.kmers


Looks good. Let's check how many k-mers in every file.

In [61]:
wc -l *.kmers

  16 aroC_10.kmers
  16 aroC_1.kmers
  16 aroC_2.kmers
  16 aroC_3.kmers
  16 aroC_4.kmers
  16 aroC_5.kmers
  16 aroC_6.kmers
  16 aroC_7.kmers
  16 aroC_8.kmers
  16 aroC_9.kmers
 160 total


Great. Let's search for files containing all these kmers to try and identify an MLST allele.

We do this by using `grep` to search (and print) any kmers in the alleles kmer file. A match only occurs in allele if we find all kmers (that is, the set of kmers found by `grep` has size 16, or equal to the set of all kmers in the alleles file).

In [80]:
reads="test/fastq/aroC-11_GCF_000940975.1_ASM94097v1_genomic.fna.reads.fq"
fqname=`basename $reads | grep -o "^[^_]*"`
echo ${fqname}

aroC-11


In [104]:
temp_dir=`mktemp -d`
#temp_dir=/tmp/tmp.wG0oJG2Gxe
for reads in test/fastq/*.fq
#for reads in test/fastq/aroC-5_GCF_000750435.1_ASM75043v1_genomic.fna.reads.fq
do
    #fq_allele_name=`basename $reads | grep -o "^[^_]*"`
    fq_allele_name=`basename $reads`
    kmer_count_part="${temp_dir}/${fq_allele_name}"
    
    # Build commands
    commands_file=`mktemp`
    for allele in *.kmers
    do
        kmer_count_out="${kmer_count_part}_${allele}.count"
        echo "grep -F -f ${allele} ${reads} -o | sort | uniq -c | wc -l > ${kmer_count_out}" >> ${commands_file}
    done
    
    #echo "Will execute commands from file ${commands_file} like:"
    #head -n 3 ${commands_file}
    command="parallel -j 10 -a ${commands_file}"
    echo "Working on ${reads} [${command}]"
    ${command}
done

(echo -e "file\tallele";
for allele in *.kmers
do
    count_unique_allele=`wc -l ${allele} | cut -d ' ' -f 1`
    for fastq_name in ${temp_dir}/*${allele}.count
    do
        fq_allele_name=`basename $fastq_name`
        count_unique_fastq=`cat ${fastq_name}`
        #echo -e "${count_unique_fastq}\t${count_unique_alleles}"
        if [ "${count_unique_fastq}" -eq "${count_unique_allele}" ]
        then
            echo -e "${fq_allele_name}\t${allele}"
        fi
    done
done) | tee kmer-match-table.tsv | column -s$'\t' -t

Working on test/fastq/aroC-11_GCF_000940975.1_ASM94097v1_genomic.fna.reads.fq [parallel -j 10 -a /tmp/tmp.FqXFvxDXtF]
Working on test/fastq/aroC-5_GCF_000750435.1_ASM75043v1_genomic.fna.reads.fq [parallel -j 10 -a /tmp/tmp.EvzgG3IoR6]
Working on test/fastq/aroC-8_GCF_000487135.1_ASM48713v1_genomic.fna.reads.fq [parallel -j 10 -a /tmp/tmp.dvqlV78uLF]
Working on test/fastq/aroC-9_GCF_000430975.2_ASM43097v2_genomic.fna.reads.fq [parallel -j 10 -a /tmp/tmp.5VfgDwxXzw]
file                                                                       allele
aroC-5_GCF_000750435.1_ASM75043v1_genomic.fna.reads.fq_aroC_5.kmers.count  aroC_5.kmers
aroC-8_GCF_000487135.1_ASM48713v1_genomic.fna.reads.fq_aroC_8.kmers.count  aroC_8.kmers
aroC-9_GCF_000430975.2_ASM43097v2_genomic.fna.reads.fq_aroC_9.kmers.count  aroC_9.kmers
